In [2]:
from ioMicro import  *


In [3]:
flds = glob.glob(r'Z:\PTK_SI14_CtrlP2_03_06_2023\H*')+glob.glob(r'Y:\PTK_SI14_CtrlP2_03_06_2023\H*')
flds = np.array(flds)[np.argsort([get_iH(fld) for fld in flds])]
tags = [os.path.basename(fld) for fld in flds]
fovs = [os.path.basename(fl) for fl in np.sort(glob.glob(flds[0]+os.sep+'*.zarr'))]

### Check all the hybes for one fov

In [4]:
fov = fovs[43]
fls = [fld+os.sep+fov for fld in flds]

In [5]:
import napari
viewer = napari.Viewer()
fl = fls[23]
im = np.array(read_im(fl))
viewer.add_image(im,name = os.path.basename(os.path.dirname(fl))+'_col0')

<Image layer 'H47R72_R97_col0' at 0x1e9f2e19820>

In [6]:
import napari
viewer = napari.Viewer()
fl = fls[23]
for fl in tqdm(fls[:]):
    im = np.array(read_im(fl)[0][11])
    viewer.add_image(im,name = os.path.basename(os.path.dirname(fl))+'_col0')
    im = np.array(read_im(fl)[1][11])
    viewer.add_image(im,name = os.path.basename(os.path.dirname(fl))+'_col1')


100%|██████████████████████████████████████████████████████████████████████████████████| 35/35 [02:42<00:00,  4.65s/it]


In [4]:
import napari
viewer = napari.Viewer()

In [48]:
fl = fls[-3]
im = read_im(fl)

viewer.add_image(im,name = os.path.basename(os.path.dirname(fl)))

<Image layer 'H60U1' at 0x19116844130>

In [5]:
fl = fls[-3]
im = np.array(read_im(fl)[-1][10])
im = im[1300:1800,600:1100]
napari.view_image(im);

In [63]:
import ioMicro
!"{ioMicro.__file__}"

The system cannot execute the specified program.


In [97]:
all_models = ['cyto',
 'nuclei',
 'cyto2',
 'bact_phase_cp',
 'bact_fluor_cp',
 'plant_cp',
 'worm_cp',
 'cyto2_omni',
 'bact_phase_omni',
 'bact_fluor_omni',
 'plant_omni',
 'worm_omni',
 'worm_bact_omni',
 'worm_high_res_omni']

In [98]:
from cellpose import models, io,utils
model='bact_phase_omni'
model = models.Cellpose(gpu=True, model_type=model)
chan = [0,0]
im_ = np.array(im,dtype=np.float32)
resc = 1
#img = (cv2.blur(im_,(2,2))-cv2.blur(im_,(300,300)))[::resc,::resc]
img = -(cv2.blur(im_,(2,2))-cv2.blur(im_,(40,40)))[::resc,::resc]

#img = -(im_-cv2.blur(im_,(30,30)))[::resc,::resc]
p1 = np.percentile(img,1)
p99=None
if p99 is None:
    p99 = np.percentile(img,99.5)
img = np.array(np.clip((img-p1)/(p99-p1),0,1),dtype=np.float32)
#masks, flows, styles, diams = model.eval(img, diameter=15, channels=chan,
#                                     min_size=10,normalize=True,omni=True)

# define parameters
mask_threshold = -1 
verbose = 0 # turn on if you want to see more output 
use_gpu = True #defined above
transparency = True # transparency in flow output
rescale=None # give this a number if you need to upscale or downscale your images
omni = True # we can turn off Omnipose mask reconstruction, not advised 
flow_threshold = 0 # default is .4, but only needed if there are spurious masks to clean up; slows down output
resample = True #whether or not to run dynamics on rescaled grid or original grid 
cluster=True # use DBSCAN clustering

masks, flows, styles,diams = model.eval(img,channels=chan,rescale=rescale,mask_threshold=mask_threshold,
                                  transparency=transparency,flow_threshold=flow_threshold,omni=omni,
                                  cluster=cluster, resample=True,verbose=verbose)

viewer = napari.view_image(1-img)
viewer.add_labels(masks)

2023-03-05 07:33:24,628 [INFO] TORCH GPU version not installed/working.
2023-03-05 07:33:24,628 [INFO] >>>> using CPU
2023-03-05 07:33:24,751 [INFO] ~~~ FINDING MASKS ~~~
2023-03-05 07:33:27,156 [INFO] >>>> TOTAL TIME 2.41 sec


<Labels layer 'masks' at 0x1f65e7a48b0>

### bits

In [102]:
im = read_im(fls[0])

In [103]:
napari.view_image(im)

Viewer(axes=Axes(visible=False, labels=True, colored=True, dashed=False, arrows=True), camera=Camera(center=(0.0, 1399.5, 1399.5), zoom=0.40883928571428574, angles=(0.0, 0.0, 90.0), perspective=0.0, interactive=True), cursor=Cursor(position=(1.0, 1.0, 0.0, 0.0), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=4, ndisplay=2, last_used=0, range=((0.0, 3.0, 1.0), (0.0, 20.0, 1.0), (0.0, 2800.0, 1.0), (0.0, 2800.0, 1.0)), current_step=(1, 10, 1400, 1400), order=(0, 1, 2, 3), axis_labels=('0', '1', '2', '3')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'im' at 0x1f66a79ee50>], scale_bar=ScaleBar(visible=False, colored=False, ticks=True, position=<Position.BOTTOM_RIGHT: 'bottom_right'>, font_size=10, unit=None), text_overlay=TextOverlay(visible=False, color=(0.5, 0.5, 0.5, 1.0), font_size=10, position=<TextOverlayPosition.TOP_LEFT: 'top_left'>, text=''), overlays=Overlays(interaction_box=InteractionBox(points=None, show=False

In [109]:
fl = fls[0]
im_dapi = np.array(read_im(fl)[-1][10])
im_dapi = im_dapi[1300:1800,600:1100]

viewer = napari.view_image(im_dapi);
im_sig = np.max(im[1][6:10],axis=0)[1300:1800,600:1100]
viewer.add_image(im_sig)


<Image layer 'im_sig' at 0x1f66a8b9760>

In [116]:
fl = fls[0]
im_dapi = np.array(read_im(fl)[-1])
fl = fls[-2]
im_dapi2 = np.array(read_im(fl)[-1])

<Image layer 'im_sig [1]' at 0x1f6cec458b0>

In [124]:
fl_ref = fls[-2]
im_dapi2 = np.array(read_im(fl_ref)[-1])
im_dapi2_ = im_dapi2[10,1300:1800,600:1100]
viewer = napari.view_image(im_dapi2_);


fl = fls[0]
im_dapi = np.array(read_im(fl)[-1])

txyz,txyzs = get_txyz(im_dapi,im_dapi2,sz_norm=20,sz=400)

from dask import array as da
im_ = da.roll(read_im(fl),txyz,axis=[1,2,3])
im_sig = np.max(im_[1][6:10],axis=0)[1300:1800,600:1100]


viewer.add_image(im_sig)

<Image layer 'im_sig' at 0x1f6cef383d0>

In [138]:
ifl = [ifl for ifl,fl in enumerate(fls) if 'U7' in os.path.basename(os.path.dirname(fl))][0]
fl = fls[ifl]
im_dapi = np.array(read_im(fl)[-1])

txyz,txyzs = get_txyz(im_dapi,im_dapi2,sz_norm=20,sz=400)

from dask import array as da
im_ = da.roll(read_im(fl),txyz,axis=[1,2,3])

icol=1
im_sig = np.max(im_[icol][6:10],axis=0)[1300:1800,600:1100]
viewer.add_image(im_sig,name = os.path.basename(os.path.dirname(fl))+'_'+str(icol))

<Image layer 'H55U7_1' at 0x1f6ba12d220>

In [108]:
class spread_decoder():
    def __init__(self,data_folders = [r'Z:\PTK_SI14_Ctrl_02_26_2023\H*'],analysis_folder=r'Z:\PTK_SI14_Ctrl_02_26_2023_analysis'):
        flds =  [fld for data_fld in data_folders for fld in glob.glob(data_fld)]
        self.analysis_folder = analysis_folder
        self.flds = flds = np.array(flds)[np.argsort([get_iH(fld) for fld in flds])]
        tags = [os.path.basename(fld) for fld in flds]
        self.fovs = [os.path.basename(fl) for fl in np.sort(glob.glob(flds[0]+os.sep+'*.zarr'))]
        self.col0Rs = col0Rs = ['R26', 'R27', 'R3', 'R4', 'R30', 'R31', 'R32', 'R8', 'R34', 'R35', 'R36', 'R37', 'R38', 'R14', 'R40', 'R41', 'R42', 'R43', 'R44', 'R45', 'R21', 'R48', 'R49', 'R50', 'R76', 'R77', 'R78', 'R79', 'R80', 'R81', 'R82', 'R83', 'R84', 'R85', 'R86', 'R87', 'R88', 'R89', 'R91', 'R67', 'R93', 'R94', 'R95', 'R96', 'R72', 'R98', 'R99', 'R100']
        dic_R_to_ifld = {}
        for ifld,fld in enumerate(self.flds):
            Rs = htag_to_Rs(os.path.basename(fld))
            for R in Rs:
                icol=0 if R in col0Rs else 1
                dic_R_to_ifld[R]=(ifld,icol,fld)
        self.dic_R_to_ifld = dic_R_to_ifld

In [117]:
dec = spread_decoder(data_folders = [r'Z:\PTK_SI14_Ctrl_02_26_2023\H*'],
                     analysis_folder=r'Z:\PTK_SI14_Ctrl_02_26_2023_analysis')

In [192]:
from dask import array as da
def load_hybe_fov_im(self,R=None,ifld=0,ifov=0,icol=0,ref=20,force=False):
    if R is not None:
        ifld,icol,_ = self.dic_R_to_ifld[R]
        self.ifld,self.icol = ifld,icol
    self.fov = self.fovs[ifov]
    self.fl = self.flds[ifld]+os.sep+self.fov
    self.fl_ref = self.flds[ref]+os.sep+self.fov
    
    def get_tag(fl): return os.path.basename(os.path.dirname(fl))
    self.drift_save = drift_save = self.analysis_folder+os.sep+'drifts'+get_tag(self.fl_ref)+'--'+get_tag(self.fl)+'--'+os.path.basename(self.fl).split('.')[0]+'.drift'
    if not os.path.exists(os.path.dirname(drift_save)):os.makedirs(os.path.dirname(drift_save))
    if (not os.path.exists(self.drift_save)) or force:
        if self.fl_ref!=getattr(self,'prev_ref',''):
            print("Reading dapi ref image:"+str(self.fl_ref))
            self.im_dapi_ref = im_dapi = np.array(read_im(self.fl_ref)[-1],dtype=np.float32)
            self.prev_ref = self.fl_ref
        print("Reading dapi image:"+str(self.fl))
        self.im_dapi = np.array(read_im(self.fl)[-1],dtype=np.float32)
        print("Computing registration...")
        txyz,txyzs = get_txyz(self.im_dapi_ref,self.im_dapi,sz_norm=20,sz=400)
        self.txyz,self.txyzs = txyz,txyzs
        pickle.dump([txyz,txyzs,self.fl,self.fl_ref],open(self.drift_save ,'wb'))
    else:
        txyz,txyzs,_,_ = pickle.load(open(self.drift_save ,'rb'))
        self.txyz,self.txyzs = txyz,txyzs
    print("Drift:"+str(self.txyz))
    
    self.imf = da.roll(read_im(self.fl),-txyz,axis=[1,2,3])
    self.im = self.imf[icol]

In [119]:

#['R47','R90']
#R21 - reimaged
12--19
26--63
93--95
73--97
24--40
39--51
29--69
7--42
25--61
55--78

In [219]:
#dec.dic_R_to_ifld
for ifld,fld in enumerate(dec.flds):
    tag = os.path.basename(fld)
    if 'U' in tag:
        R = 'U'+tag.split('U')[-1]
        dec.dic_R_to_ifld[R]=[ifld,1,fld]

In [175]:
dic_Xh = {}

In [188]:
dic_ims = {}

In [233]:
#for R in ['R12','R19','U1']:
#for R in ['R26','R63','U2']:
for R in ['R93','R95','U3']:
    load_hybe_fov_im(dec,R=R,ifov=43,ref=20,force=False)
    slice_cell = tuple([slice(4,10),slice(1275,1775),slice(700,1200)])
    im_ = np.array(dec.im[slice_cell],dtype=np.float32)
    th_ = np.std(im_)*3+np.median(im_)
    Xh = get_local_max(im_,th_fit=th_,im_raw=im_,delta=2,delta_fit=3)
    dic_Xh[R] = Xh
    dic_ims[R]=im_
    


Drift:[-1 23  7]
Drift:[  0  17 -87]
Drift:[  0  37 -82]


In [234]:
viewer = napari.Viewer()
cols=['r','g','y']
for iR,R in enumerate(['R93','R95','U3']):
    Xh = dic_Xh[R].copy()
    zc,xc,yc,bk,a,habs,hn,h = Xh.T
    keep = np.argsort(a)[::-1][:500]
    Xh = Xh[keep]
    zc,xc,yc,bk,a,habs,hn,h = Xh.T
    size = (a-np.min(a))/(np.max(a)-np.min(a))*10
    viewer.add_image(np.max(dic_ims[R],axis=0),name=R)
    viewer.add_points(Xh[:,1:3],size=size,face_color=[0,0,0,0],edge_color=cols[iR%len(cols)],name=R)
    

In [195]:
viewer = napari.Viewer()
for R in dic_ims:
    viewer.add_image(dic_ims[R],name=R)

In [184]:
dec.fl

'Z:\\PTK_SI14_Ctrl_02_26_2023\\H1R1_R26\\Conv_zscan__43.zarr'

In [158]:
th_ = np.std(im_)*3+np.median(im_)
Xh = get_local_max(im_,th_fit=th_,im_raw=im_,delta=2,delta_fit=4)

<Points layer 'Points' at 0x1ec59523640>

In [148]:
import napari
napari.view_image(im_)
        

Viewer(axes=Axes(visible=False, labels=True, colored=True, dashed=False, arrows=True), camera=Camera(center=(0.0, 249.5, 249.5), zoom=2.3313, angles=(0.0, 0.0, 90.0), perspective=0.0, interactive=True), cursor=Cursor(position=(1.0, 1.0, 0.0), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=3, ndisplay=2, last_used=0, range=((0.0, 4.0, 1.0), (0.0, 500.0, 1.0), (0.0, 500.0, 1.0)), current_step=(2, 250, 250), order=(0, 1, 2), axis_labels=('0', '1', '2')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'im_' at 0x1ec5c368fd0>], scale_bar=ScaleBar(visible=False, colored=False, ticks=True, position=<Position.BOTTOM_RIGHT: 'bottom_right'>, font_size=10, unit=None), text_overlay=TextOverlay(visible=False, color=(0.5, 0.5, 0.5, 1.0), font_size=10, position=<TextOverlayPosition.TOP_LEFT: 'top_left'>, text=''), overlays=Overlays(interaction_box=InteractionBox(points=None, show=False, show_handle=False, show_vertices=False, selection_b

In [107]:
dic_R_to_ifld['R9']

(9, 1, 'Z:\\PTK_SI14_Ctrl_02_26_2023\\H9R9_R34')

In [92]:
htag_to_Rs(fld)

['R69', 'R94']

In [72]:
let='0'
let.isnumeric()

True